# `rosalie` usage examples

## Simple example

### Import dependencies

- Import `rosalie` and other dependencies to specify your evaluator methods.

In [1]:
from scipy.stats import mannwhitneyu
from statsmodels.stats.weightstats import ttest_ind

import rosalie as ro

# High resulution plots
%config InlineBackend.figure_format ='retina'

### Load data

- You can load data any way and from anywhere you want.

- The only requirement on the data is that there is an `id` column and a column with at least one metric you want to perform the evaluation for.

- Below I use `rosalie`'s data reader, which will eventually make it very easy to read customer and restaurant level JET data. For now it reads a small customer-level dataset that I then process further to get into the needed shape.

- Specifying a `cache_path` is optional, but automatically stores data locally if it doesn't exist already to allow for faster reading in the future.

In [2]:
CACHE_PATH = f"/Users/fabian.gunzinger/tmp/rosalie/customer.csv"

df = (
    ro.DataReader().load_data("customer", cache_path=CACHE_PATH)
    .groupby('id')
    .order_price
    .mean()
    .dropna()
    .reset_index()
)
df.head()

Reading data from cache...


,id,order_price
0,JE:IE:1000053,18.851200
1,JE:IE:1000071,25.943750
2,JE:IE:1000093,25.000000
3,JE:IE:1000153,17.041111
4,JE:IE:100021,21.775000


### Define evaluation functions

- You can specify any evaluation functions you want.

- The only requirements are that they take a Dataframe and a metric as arguments, and return a p-value.

In [7]:
def welch(df, metric):
    """Return p-value of Welch's t-test."""
    control_sample = df[df["assignments"] == "control"][metric]
    variant_sample = df[df["assignments"] == "treatment"][metric]
    _, p, _ = ttest_ind(control_sample, variant_sample, usevar="unequal")
    return p


def mmw(df, metric):
    """
    Returns p-value of Mann-Whitney-Wilcoxon rank-sum test.
    """
    control_sample = df[df["assignments"] == "control"][metric]
    variant_sample = df[df["assignments"] == "treatment"][metric]
    _, p = mannwhitneyu(control_sample, variant_sample, alternative="two-sided")
    return p

### Run simulator

- To run the simulator, pass the data and the evaluator functions.

- There are a number of optional arguments we will use below, but the class has sensible defaults.

- The `Simulator` returns a `Result` object which contains the evaluation results and allows us to plot them easily. To see the plot, use the `plot()` method.

In [14]:
eval = ro.Simulator(
    df=df,
    metrics=['order_price'],
    evaluators=[welch, mmw],
    )
result = eval.run()
result.plot()

INFO - Initializing Simulator with specified evaluators: ['welch', 'mmw']
INFO - Initializing Simulator with specified evaluators: ['welch', 'mmw']
INFO - Generating datasets...
INFO - Generating datasets...
100%|██████████| 20/20 [00:42<00:00,  2.14s/it]
INFO - Evaluating experiments...
INFO - Evaluating experiments...
100%|██████████| 2000/2000 [00:51<00:00, 38.93it/s] 


alt.VConcatChart(...)

To access the precise results, use the `data` attribute.

In [18]:
result.data

,metric,evaluator,sample_size,mdes,power
0,order_price,mmw,100,0.01,0.10
1,order_price,mmw,7369,0.01,0.14
2,order_price,mmw,14637,0.01,0.30
3,order_price,mmw,21906,0.01,0.44
4,order_price,mmw,29174,0.01,0.60
5,order_price,mmw,36443,0.01,0.72
6,order_price,mmw,43711,0.01,0.78
7,order_price,mmw,50980,0.01,0.82
8,order_price,mmw,58248,0.01,0.82
9,order_price,mmw,65517,0.01,0.92


## More complex example -- comparing CUPED implementations

### Import dependencies

In [8]:
import logging

import numpy as np
import pandas as pd
import statsmodels.api as sm

from causaljet.experiment_evaluation.models import Cuped
import rosalie as ro

# Silence info logging of root logger to silence Cuped logging
logging.getLogger().setLevel(logging.WARNING)


%config InlineBackend.figure_format ='retina'
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load data

In [5]:
UNIT_LEVEL = "customer"
UNIT_ID = 'id'
METRICS = ['order_price', 'gmv']
PRE_PERIOD = '1 Jan 2023', '31 May 2023'
POST_PERIOD ='1 Jun 2023', ' 31 Aug 2023'

CACHE_PATH = f'/Users/fabian.gunzinger/tmp/rosalie/{UNIT_LEVEL}.csv'

df = (
    ro.DataReader().load_data(UNIT_LEVEL, cache_path=CACHE_PATH)
    .pipe(ro.add_artificial_gmw)
    .pipe(ro.create_pre_post_data, 
          id_col=UNIT_ID,
          metrics=METRICS,
          pre_period=PRE_PERIOD,
          post_period=POST_PERIOD)
)
ro.data_info(df)

Reading data from cache...
Shape: (62279, 5)
Units: 62,279
                  id  order_price        gmv  order_price_pre    gmv_pre
53943  JE:IE:1000071    22.625000  25.209507        27.049999  24.457850
53944  JE:IE:1000153    14.245000  24.127012        17.390625  25.436335
53945   JE:IE:100021    12.950000  21.287273        30.600000  25.812185
53947  JE:IE:1000294    23.299999  25.023067        29.455999  24.861668
53948  JE:IE:1000373    27.950001  27.261032        26.139999  25.673666


### Define evaluation functions

In [6]:
def causal_jet_cuped(df, metric):
    """Run Causal Jet CUPED implementation and return p-value.

    Because data is already pre-processed, we only need to supply the following:
    - A cross-section dataframe with `metric` and `metric_pre` columns to `ass_w_cov_panel_df`
    - The metric name to `metric_name`
    - The unit identifier to `unit_identifier`

    All other parameters can be left as default.
    """
    result = Cuped(
        ass_w_cov_panel_df=df,
        metric_name=metric,
        unit_identifier=UNIT_ID,
        cluster_identifier=UNIT_ID,
        is_treated_col='is_treated',
        weight_col='assignments_freq',
        additional_regressors=[],
        start_date=None,
        date_identifier=None,
        lookback=None,
    )._get_results()
    return result.pvalues[1]


def traditional_cuped(df, metric):
    """Run traditional CUPED and return p-value."""
    
    def _cuped_adjusted_metric(df, metric, metric_pre):
        y = df[metric].values
        x = df[metric_pre].values
        valid_indices = (~np.isnan(y)) & (~np.isnan(x))
        y_valid, x_valid = y[valid_indices], x[valid_indices]
        m = np.cov(y_valid, x_valid)
        theta = m[0, 1] / m[1, 1]
        return (y - (x - np.nanmean(x)) * theta)

    # Perform experiment evaluation and return p-value
    # (Use WLS to be consistent with CausalJet)
    y = _cuped_adjusted_metric(df, metric, f"{metric}_pre")
    x = sm.add_constant(df["is_treated"].astype(float))
    w = df["assignments_freq"]
    model = sm.WLS(endog=y, exog=x, weights=w)
    results = model.fit()
    return results.pvalues["is_treated"]

In [9]:
evaluators = [traditional_cuped, causal_jet_cuped]

simulator = ro.Simulator(
    df=df,
    evaluators=evaluators,
    # preprocessors=None,
    baseline_evaluator="wls",
    id_col=UNIT_ID,
    # time_col="timeframe",
    metrics=METRICS,
    # sample_min=None,
    # sample_max=None,
    num_steps=5,
    # sample_timestamps=False,
    num_runs=5,
    # random_seed=2312,
    mdes=[0.01],
    # alpha=0.05,
    )

result = simulator.run()
print(result.data.head())
result.plot()

INFO - Initializing Simulator with specified evaluators: ['wls', 'traditional_cuped', 'causal_jet_cuped']


2023-11-02 14:54:02,551 | rosalie.simulator    | simulator.py:295 | INFO     | Initializing Simulator with specified evaluators: ['wls', 'traditional_cuped', 'causal_jet_cuped']


INFO - Generating datasets...


2023-11-02 14:54:02,552 | rosalie.simulator    | simulator.py:110 | INFO     | Generating datasets...


100%|██████████| 5/5 [00:00<00:00,  8.73it/s]
INFO - Evaluating experiments...


2023-11-02 14:54:03,126 | rosalie.simulator    | simulator.py:127 | INFO     | Evaluating experiments...


100%|██████████| 150/150 [00:00<00:00, 178.82it/s]


  metric         evaluator  sample_size  mdes  power
0    gmv  causal_jet_cuped          100  0.01    0.0
1    gmv  causal_jet_cuped        15645  0.01    1.0
2    gmv  causal_jet_cuped        31190  0.01    1.0
3    gmv  causal_jet_cuped        46734  0.01    1.0
4    gmv  causal_jet_cuped        62279  0.01    1.0


alt.VConcatChart(...)